# Desafio IA 2020 - PLN
## Serpro

### Participante: Andréa Jesus dos Santos
### Equipe: Leão

In [1]:
#Import libraries
from sklearn import datasets
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn import metrics #for accuracy calculation
#import os
import pandas as pd 
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import ShuffleSplit

import emocoes_texto_classificacao

from nltk.corpus import stopwords
from string import punctuation

import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.datasets as skldata
import nltk


nltk.download('rslp')

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\Deinha\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

## 1. Carregar Dados 

In [2]:
caminhoDatabase = "C:\\Users\\Deinha\\Documents\\Serpro\\DesafioIA\\PLN\\Codigo\\Dataset\\"

TREINO_PATH = caminhoDatabase + "treino.csv"
TESTE_PATH = caminhoDatabase + "teste-sem-classe.csv"

#Dados de Treino
#treino_data = skldata.load_files(TREINO_PATH, shuffle=False, load_content=True, encoding="utf-8")

dfTreino_original = pd.read_csv(TREINO_PATH, sep=",", encoding='utf-8')
#nomes_classificacao_treino = treino_data.target_names
#label2idx = {label: idx for idx, label in enumerate(nomes_classificacao_treino)}

#Tipos de Classificação do Texto
'''
print("Nome classificação: ", nomes_classificacao_treino)
print("Classificação: ", label2idx)
print("Treino target: ", treino_data.target)
'''

#Dados de Teste
#teste_data = skldata.load_files(TESTE_PATH, shuffle=False, load_content=True, encoding="utf-8")
dfTeste_original = pd.read_csv(TESTE_PATH, sep=",", encoding='utf-8')

#Nomes das Colunas
#rotulos_colunas = dfTreino_original.columns[0:3]
#print(rotulos_colunas)

#path = "../dataset/"
#df = emocoes_texto_classificacao.format_dataset(path)
#df.head()

print("Fim do carregamento de dados")

Fim do carregamento de dados


## 2. Pré processamento

In [3]:
#Exclusão em memória do campo ID
#dfTreino = dfTreino_original.drop(labels=["id"], axis=1, inplace=False)
#dfTeste = dfTeste_original.drop(labels=["id"], axis=1, inplace=False)

dfTreino = dfTreino_original
dfTeste = dfTeste_original

#padronozação para letras em minúsculo
#remoção de stopwords, caracteres especiais, números e radicais
dfTreino_tratado = emocoes_texto_classificacao.text_preprocess(dfTreino)
dfTeste_tratado = emocoes_texto_classificacao.text_preprocess(dfTeste, "")

print("=-=-Tratada=-=-")
print(dfTreino_tratado.head(10))

print("Fim do pré processamento")

=-=-Tratada=-=-
   id                                              texto    classe
0   0  lul diz sen maior resolv problem presid diz de...    neutro
1   1  adolesc mort ouv music alt est unid adolesc  a...  tristeza
2   2  core sul insinu hack lig core nort atac intern...    neutro
3   3  mama foc sel filhot recemnasc belg filhot nasc...   alegria
4   4  adolesc  ano sum ach mort manau segund polic v...  tristeza
5   5  bomb localiz corp jov  ano rio mat gross cas r...  tristeza
6   6  medic tent explic orig caibr noturn episodi co...  surpresa
7   7  govern anunc liberaca r  bilho agricult famili...   alegria
8   8  sindicat cuban celebr  ano revoluca dia trabal...    neutro
9   9  eclips ajud cient identific assinat planet ter...   alegria
Fim do pré processamento


# 3. Seperar dados de treino e de teste

In [4]:
x_train, x_test, y_train, y_test = train_test_split(dfTreino_tratado['texto'].values, 
                                                    dfTreino_tratado['classe'].values, 
                                                    test_size=0.20,
                                                    random_state=10)

print('Tamanho do dataset de treino: {}'.format(len(y_train)))
print('Tamanho do dataset de teste: {}'.format(len(y_test)))

print("Fim da separação de dados")

Tamanho do dataset de treino: 1360
Tamanho do dataset de teste: 340
Fim da separação de dados


# 4. Text enconding

In [5]:
stopword = emocoes_texto_classificacao.get_stopwords()

tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=(1,2),
                        stop_words=stopword,
                        max_df=0.8,
                        max_features=100)
                        
features_train = tfidf.fit_transform(x_train)
labels_train = y_train

features_test = tfidf.transform(x_test)
labels_test = y_test

print(features_train.shape)
print(features_test.shape)

print("Fim do text encoding")

(1360, 100)
(340, 100)
Fim do text encoding


# 5. Model

In [6]:
print("Início do modelo")
C = [.001, .01, .1, 1, 10]
degree = [1, 2, 3, 4, 5]
gamma = [0.001, 0.01, 0.1, 1, 10, 100]
probability = [True, False]
kernels = ['linear', 'poly', 'rbf']

param_grid = {'C': C, 'kernel':kernels, 'probability':probability, 'gamma':gamma, 'degree':degree}

print("Fim do modelo SVM")
svc = svm.SVC(random_state=8)

print("Fim do modelos GridSearch")
grid_search = GridSearchCV(estimator=svc, 
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=10,
                           verbose=1,
                           n_jobs=-1)

print("Fim do modelos Grid Search Fit")
grid_search.fit(features_train, labels_train)

print("Melhores hiper parametros:")
print(grid_search.best_params_)
print("Acurácia:")
print(grid_search.best_score_)


print("Fim do modelo")

Início do modelo
Fim do modelo SVM
Fim do modelos GridSearch
Fim do modelos Grid Search Fit
Fitting 10 folds for each of 900 candidates, totalling 9000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   14.9s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   39.0s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done 3192 tasks      | elapsed: 10.1min
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed: 12.9min
[Parallel(n_jobs=-1)]: Done 4992 tasks      | elapsed: 16.2min
[Parallel(n_jobs=-1)]: Done 6042 tasks      | elapsed: 20.2min
[Parallel(n_jobs=-1)]: Done 7192 tasks      | elapsed: 24.2min
[Parallel(n_jobs=-1)]: Done 8442 tasks      | elapsed: 36.6min
[Parallel(n_jobs=-1)]: Done 9000 out of 9000 | elapsed: 39.1min finished
C:\ProgramData\Anaconda3\lib\site-pa

Melhores hiper parametros:
{'C': 10, 'degree': 1, 'gamma': 0.01, 'kernel': 'rbf', 'probability': True}
Acurácia:
0.4852941176470588
Fim do modelo


# 6. Resultados

In [8]:
print("Teste")

best_model = grid_search.best_estimator_
best_model.fit(features_train, labels_train)

y_predict = best_model.predict(features_test)
print("Acurácia: {}".format(accuracy_score(labels_test, y_predict)))
print(classification_report(labels_test,y_predict))
print(confusion_matrix(labels_test, y_predict, labels=['alegria', 'raiva']))

#TODO: Gerar a tabela com o resultado alcançado

print("Fim do resultados")

Teste
Acurácia: 0.45
              precision    recall  f1-score   support

     alegria       0.00      0.00      0.00        32
    desgosto       0.38      0.18      0.24        33
        medo       1.00      0.08      0.14        52
      neutro       0.43      0.96      0.59        91
       raiva       0.00      0.00      0.00        14
    surpresa       0.56      0.21      0.30        48
    tristeza       0.47      0.66      0.55        70

   micro avg       0.45      0.45      0.45       340
   macro avg       0.40      0.30      0.26       340
weighted avg       0.48      0.45      0.36       340

[[0 0]
 [0 0]]
Fim do resultados


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# 7. Extras - com parâmetros pré-definidos

Para rodar esses resultados, é necessário setar o parametro ind_stemmer em notebooks/emocoes_texto_classificacao.py como 1 (com stemmer) ou 0 (sem stemmer) e rodar novamente o código


In [9]:
print('Com stemmer')
best_model = svm.SVC(kernel='linear', C=.3, probability=True)
best_model.fit(features_train, labels_train)
y_predict = best_model.predict(features_test)

print('C=0.3')
print("Acurácia: {}".format(accuracy_score(labels_test, y_predict)))
print(classification_report(labels_test,y_predict))
print(confusion_matrix(labels_test, y_predict, labels=['alegria', 'raiva']))

best_model = svm.SVC(kernel='linear', C=1.0, probability=True)
best_model.fit(features_train, labels_train)
y_predict = best_model.predict(features_test)

print('\nC=1.0')
print("Acurácia: {}".format(accuracy_score(labels_test, y_predict)))
print(classification_report(labels_test,y_predict))
print(confusion_matrix(labels_test, y_predict, labels=['alegria', 'raiva']))

Com stemmer
C=0.3
Acurácia: 0.45588235294117646
              precision    recall  f1-score   support

     alegria       0.00      0.00      0.00        32
    desgosto       0.36      0.24      0.29        33
        medo       0.64      0.13      0.22        52
      neutro       0.44      0.92      0.60        91
       raiva       0.00      0.00      0.00        14
    surpresa       0.48      0.25      0.33        48
    tristeza       0.49      0.63      0.55        70

   micro avg       0.46      0.46      0.46       340
   macro avg       0.34      0.31      0.28       340
weighted avg       0.42      0.46      0.38       340

[[0 0]
 [0 0]]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



C=1.0
Acurácia: 0.4852941176470588
              precision    recall  f1-score   support

     alegria       0.14      0.03      0.05        32
    desgosto       0.36      0.36      0.36        33
        medo       0.54      0.29      0.37        52
      neutro       0.51      0.87      0.64        91
       raiva       0.75      0.21      0.33        14
    surpresa       0.44      0.29      0.35        48
    tristeza       0.51      0.59      0.55        70

   micro avg       0.49      0.49      0.49       340
   macro avg       0.46      0.38      0.38       340
weighted avg       0.46      0.49      0.44       340

[[1 0]
 [0 3]]
